In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


In [2]:
data = pd.read_csv("dataset.csv")

In [3]:
data = shuffle(data, random_state=22)

In [4]:
data = data.drop(columns=['app_id', 'app_name', 'review_votes'])


In [5]:
train_size = int(len(data) * .8)
print(f'Train size: {train_size}')
print(f'Test size: {len(data) - train_size} ')

Train size: 5133684
Test size: 1283422 


In [6]:
data['review_score'] = data['review_score'].apply(lambda x : 0 if x < 0 else x)


In [7]:
from tensorflow.keras.preprocessing import text

class TextPreprocessor(object):
    def __init__(self, vocab_size):
        self._vocab_size = vocab_size
        self._tokenizer = None

    def create_tokenizer(self, text_list):
        tokenizer = text.Tokenizer(num_words=self._vocab_size)
        tokenizer.fit_on_texts(text_list)
        self._tokenizer = tokenizer

    def transform_text(self, text_list):
        text_matrix = self._tokenizer.texts_to_matrix(text_list)
        return text_matrix


train_rv = data['review_text'].values[:train_size]
test_rv = data['review_text'].values[train_size:]


VOCAB_SIZE = 400

processor = TextPreprocessor(VOCAB_SIZE)
for i in range(len(train_rv)):
    train_rv[i] = str(train_rv[i])
for i in range(len(test_rv)):
    test_rv[i] = str(test_rv[i])
processor.create_tokenizer(train_rv)


body_train = processor.transform_text(train_rv)
body_test = processor.transform_text(test_rv)

In [8]:
len(body_train[0])

400

In [9]:
import pickle
with open('./processor_state.pkl', 'wb') as f:
    pickle.dump(processor, f)

In [10]:
def create_model(vocab_size):
    model = Sequential()
    model.add(Dense(50, input_shape=(vocab_size,), activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model(VOCAB_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                20050     
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 21,351
Trainable params: 21,351
Non-trainable params: 0
_________________________________________________________________


In [11]:



model.fit(body_train, data['review_score'][:train_size].values, epochs=3, batch_size=128, validation_split=0.1)




C:\Users\aliam\AppData\Local\Temp\ipykernel_10500\412868314.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  model.fit(body_train, data['review_score'][:train_size].values, epochs=3, batch_size=128, validation_split=0.1)


Epoch 1/3
36097/36097 [==============================] - 57s 2ms/step - loss: 0.3311 - accuracy: 0.8606 - val_loss: 0.3253 - val_accuracy: 0.8635
Epoch 2/3
36097/36097 [==============================] - 41s 1ms/step - loss: 0.3211 - accuracy: 0.8652 - val_loss: 0.3223 - val_accuracy: 0.8650
Epoch 3/3
36097/36097 [==============================] - 41s 1ms/step - loss: 0.3186 - accuracy: 0.8663 - val_loss: 0.3220 - val_accuracy: 0.8646


In [12]:
model.evaluate(body_test, data['review_score'][train_size:].values, batch_size=128)

C:\Users\aliam\AppData\Local\Temp\ipykernel_10500\3173960158.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  model.evaluate(body_test, data['review_score'][train_size:].values, batch_size=128)


10027/10027 [==============================] - 10s 943us/step - loss: 0.3205 - accuracy: 0.8653


[0.32045960426330566, 0.8652610182762146]

In [ ]:
model.save('steam_review_predictor.h5')